In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
import sys
import importlib
sys.path.append('..')
import preprocessing.pipeline
importlib.reload(preprocessing.pipeline)
from preprocessing.pipeline import get_preprocessor

In [2]:
train = pd.read_csv("../data/train_transformed.csv")
test = pd.read_csv("../data/test.csv")

X = train.drop("SalePrice", axis=1)
y = train["SalePrice"]

In [4]:
model = Pipeline(steps=[
  ("regressor", RandomForestRegressor(n_estimators=100, random_state=42))
])

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kf, scoring="neg_root_mean_squared_error")

print("RMSE per fold:", -scores)
print("Average RMSE:", -scores.mean())

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/andrewjowe/opt/anaconda3/envs/STA160/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/andrewjowe/opt/anaconda3/envs/STA160/lib/python3.12/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/andrewjowe/opt/anaconda3/envs/STA160/lib/python3.12/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/Users/andrewjowe/opt/anaconda3/envs

In [ ]:
model.fit(X, y)
predictions = model.predict(test)

# Create submission
submission = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": predictions
})

submission.to_csv("submission.csv", index=False)